In [1]:
from warnings import filterwarnings
filterwarnings("ignore")


In [2]:
import pandas as pd
import json
import urllib
import requests

In [3]:
import numpy as np

In [4]:
html = urllib.request.urlopen('https://hacker-news.firebaseio.com/v0/item/2169746.json?print=pretty')
json.loads(html.read())


{'by': 'timrobinson',
 'id': 2169746,
 'parent': 2169669,
 'text': "After being down all day yesterday, the site's now working for me.<p>Not much crime in my small part of south-east London, except for an ominous 'other crime' at the end of the street...",
 'time': 1296642293,
 'type': 'comment'}

In [7]:
x = urllib.request.urlopen('https://hacker-news.firebaseio.com/v0/maxitem.json?print=pretty')
max_item = int(x.read())
print(max_item)

23990757


In [9]:
data = []
number_of_entries = 1000

min_item = max_item - number_of_entries
count = 0


for i in range(min_item, max_item):
    html = urllib.request.urlopen('https://hacker-news.firebaseio.com/v0/item/' + str(i) + '.json')
    data.append(json.loads(html.read()))
    count += 1
    if count % 100 == 0:
        print(f"Loaded {count} rows")

Loaded 100 rows
Loaded 200 rows
Loaded 300 rows
Loaded 400 rows
Loaded 500 rows
Loaded 600 rows
Loaded 700 rows
Loaded 800 rows
Loaded 900 rows
Loaded 1000 rows


In [10]:
print (data[0])

{'by': 'nix23', 'id': 23989757, 'kids': [23989872, 23989888, 23989921], 'parent': 23989181, 'text': '&gt;There is a lot of implied ill-will on HN<p>So true, since Microsoft wants just the best for Humankind and is overall just a nice Company.', 'time': 1596046005, 'type': 'comment'}


In [11]:
data = [i for i in data if i is not None]


In [12]:
from pandas.io.json import json_normalize
df = pd.DataFrame.from_dict(data)
df.head(10)

,by,id,kids,parent,text,time,type,descendants,score,title,url,deleted,dead
0,nix23,23989757,"[23989872, 23989888, 23989921]",23989181.0,&gt;There is a lot of implied ill-will on HN<p...,1596046005,comment,NaN,NaN,NaN,NaN,NaN,NaN
1,aliswe,23989758,NaN,23983330.0,Yes that being said the demo is an old version.,1596046007,comment,NaN,NaN,NaN,NaN,NaN,NaN
2,DeonPenny,23989759,NaN,23989527.0,I&#x27;d do 27 months for millions of dollars....,1596046009,comment,NaN,NaN,NaN,NaN,NaN,NaN
3,blopeur,23989760,NaN,NaN,NaN,1596046016,story,0.0,2.0,GPUs for Science 2020,https://www.nersc.gov/users/training/gpus-for-...,NaN,NaN
4,msla,23989761,NaN,23989383.0,I just described how Minitel was completely di...,1596046021,comment,NaN,NaN,NaN,NaN,NaN,NaN
5,Glench,23989762,NaN,NaN,NaN,1596046021,story,0.0,1.0,"To recharge America’s job market, build a gree...",https://fortune.com/2020/07/29/climate-change-...,NaN,NaN
6,aspenmayer,23989763,[23989804],NaN,NaN,1596046028,story,1.0,1.0,Phone backup and a new storage manager in the ...,https://blog.google/products/google-one/free-f...,NaN,NaN
7,pc86,23989764,NaN,23987345.0,"&gt; See the problem now?<p>No, because I am a...",1596046032,comment,NaN,NaN,NaN,NaN,NaN,NaN
8,dheera,23989765,"[23989830, 23989853, 23989813, 23989822]",23989527.0,"Honest question here. Why are we, the taxpayer...",1596046041,comment,NaN,NaN,NaN,NaN,NaN,NaN
9,UncleMeat,23989766,"[23990081, 23989988]",23989664.0,&gt; You have to bake the security into your w...,1596046041,comment,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
print(df.shape)


(1000, 13)


In [14]:
df.columns

Index(['by', 'id', 'kids', 'parent', 'text', 'time', 'type', 'descendants',
       'score', 'title', 'url', 'deleted', 'dead'],
      dtype='object')

In [15]:
df_new = df.drop(columns = ['deleted', 'dead', 'descendants', 'score', 'kids', 'parent', 'title', 'url'])
df_new.head(10)


,by,id,text,time,type
0,nix23,23989757,&gt;There is a lot of implied ill-will on HN<p...,1596046005,comment
1,aliswe,23989758,Yes that being said the demo is an old version.,1596046007,comment
2,DeonPenny,23989759,I&#x27;d do 27 months for millions of dollars....,1596046009,comment
3,blopeur,23989760,NaN,1596046016,story
4,msla,23989761,I just described how Minitel was completely di...,1596046021,comment
5,Glench,23989762,NaN,1596046021,story
6,aspenmayer,23989763,NaN,1596046028,story
7,pc86,23989764,"&gt; See the problem now?<p>No, because I am a...",1596046032,comment
8,dheera,23989765,"Honest question here. Why are we, the taxpayer...",1596046041,comment
9,UncleMeat,23989766,&gt; You have to bake the security into your w...,1596046041,comment


In [16]:
df_new['text'][0]


'&gt;There is a lot of implied ill-will on HN<p>So true, since Microsoft wants just the best for Humankind and is overall just a nice Company.'

In [17]:
df_comments = df_new[df_new['type'] == 'comment']
df_comments['text']


0      &gt;There is a lot of implied ill-will on HN<p...
1        Yes that being said the demo is an old version.
2      I&#x27;d do 27 months for millions of dollars....
4      I just described how Minitel was completely di...
7      &gt; See the problem now?<p>No, because I am a...
                             ...                        
994    Can&#x27;t we just shoot a orbital space laser...
995    Under figure 1 it says &quot;Comparisons are n...
996    Would you please review the site guidelines an...
997    CYCLADES was really influencial on that (<a hr...
998    Forgive my ignorance on the subject but if it&...
Name: text, Length: 872, dtype: object

In [9]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')

import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


df_comments['clean_text']=df_comments['text'].map(lambda s:preprocess(s)) 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


NameError: ignored

In [19]:
df_comments.head(10)

,by,id,text,time,type,clean_text
0,nix23,23989757,&gt;There is a lot of implied ill-will on HN<p...,1596046005,comment,lot implied ill hnso true since microsoft want...
1,aliswe,23989758,Yes that being said the demo is an old version.,1596046007,comment,yes said demo old version
2,DeonPenny,23989759,I&#x27;d do 27 months for millions of dollars....,1596046009,comment,months millions dollars might even thousands d...
4,msla,23989761,I just described how Minitel was completely di...,1596046021,comment,described minitel completely different interne...
7,pc86,23989764,"&gt; See the problem now?<p>No, because I am a...",1596046032,comment,see problem free speech advocate thing free sp...
8,dheera,23989765,"Honest question here. Why are we, the taxpayer...",1596046041,comment,honest question taxpayers paying months free h...
9,UncleMeat,23989766,&gt; You have to bake the security into your w...,1596046041,comment,bake security workflows processes code want ma...
10,gmfawcett,23989767,Sorry to hear about the flashbacks. Having wri...,1596046044,comment,sorry hear flashbacks written couple dreadful ...
11,beervirus,23989768,Strange. Works beautifully on iOS.,1596046049,comment,strange works beautifully ios
12,awillen,23989769,You&#x27;re obviously coming in with an anti-P...,1596046054,comment,obviously coming anti bent interested pull sta...


In [20]:
df_comments['by'].value_counts()

valuearb         8
munk-a           8
skbt             6
oblio            6
jandrese         6
                ..
Simulacra        1
jaybeeayyy       1
macspoofing      1
searchableguy    1
nightfly         1
Name: by, Length: 589, dtype: int64

In [21]:
df_comments.shape


(872, 6)

In [22]:
df_comments['text'] = df_comments['text'].astype(str)
df_comments['text'][:10]


0     &gt;There is a lot of implied ill-will on HN<p...
1       Yes that being said the demo is an old version.
2     I&#x27;d do 27 months for millions of dollars....
4     I just described how Minitel was completely di...
7     &gt; See the problem now?<p>No, because I am a...
8     Honest question here. Why are we, the taxpayer...
9     &gt; You have to bake the security into your w...
10    Sorry to hear about the flashbacks. Having wri...
11                   Strange. Works beautifully on iOS.
12    You&#x27;re obviously coming in with an anti-P...
Name: text, dtype: object

In [23]:
df_comments['clean_text'] = df_comments['clean_text'].astype(str)
df_comments['clean_text'][:10]


0     lot implied ill hnso true since microsoft want...
1                             yes said demo old version
2     months millions dollars might even thousands d...
4     described minitel completely different interne...
7     see problem free speech advocate thing free sp...
8     honest question taxpayers paying months free h...
9     bake security workflows processes code want ma...
10    sorry hear flashbacks written couple dreadful ...
11                        strange works beautifully ios
12    obviously coming anti bent interested pull sta...
Name: clean_text, dtype: object

In [24]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
!pip install vaderSentiment

In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


In [19]:
def get_compund_score(text):
        score = analyzer.polarity_scores(text)
        str(text)
        return score['compound']


In [20]:
get_compund_score(df_comments['text'][0])

NameError: ignored

In [29]:
get_compund_score(df_comments['clean_text'][0])

0.7906

In [30]:
df_comments['vader_score'] = df_comments['text'].apply(get_compund_score)


In [31]:
df_comments['clean_vader_score'] = df_comments['clean_text'].apply(get_compund_score)


In [32]:
df_comments.head(10)

,by,id,text,time,type,clean_text,vader_score,clean_vader_score
0,nix23,23989757,&gt;There is a lot of implied ill-will on HN<p...,1596046005,comment,lot implied ill hnso true since microsoft want...,0.8689,0.7906
1,aliswe,23989758,Yes that being said the demo is an old version.,1596046007,comment,yes said demo old version,0.4019,0.4019
2,DeonPenny,23989759,I&#x27;d do 27 months for millions of dollars....,1596046009,comment,months millions dollars might even thousands d...,0.0000,0.0000
4,msla,23989761,I just described how Minitel was completely di...,1596046021,comment,described minitel completely different interne...,0.0000,0.0000
7,pc86,23989764,"&gt; See the problem now?<p>No, because I am a...",1596046032,comment,see problem free speech advocate thing free sp...,0.7602,0.9040
8,dheera,23989765,"Honest question here. Why are we, the taxpayer...",1596046041,comment,honest question taxpayers paying months free h...,0.8951,0.6808
9,UncleMeat,23989766,&gt; You have to bake the security into your w...,1596046041,comment,bake security workflows processes code want ma...,0.4019,0.4215
10,gmfawcett,23989767,Sorry to hear about the flashbacks. Having wri...,1596046044,comment,sorry hear flashbacks written couple dreadful ...,0.8244,0.2263
11,beervirus,23989768,Strange. Works beautifully on iOS.,1596046049,comment,strange works beautifully ios,0.4404,0.4404
12,awillen,23989769,You&#x27;re obviously coming in with an anti-P...,1596046054,comment,obviously coming anti bent interested pull sta...,0.7003,0.5423


In [33]:
df_comments['sentiment'] = df_comments['vader_score'].apply(lambda c: 'positive' if c >=0 else 'negative')


In [34]:
df_comments['clean_sentiment'] = df_comments['clean_vader_score'].apply(lambda c: 'positive' if c >=0 else 'negative')


In [35]:
df_comments.head(25)

,by,id,text,time,type,clean_text,vader_score,clean_vader_score,sentiment,clean_sentiment
0,nix23,23989757,&gt;There is a lot of implied ill-will on HN<p...,1596046005,comment,lot implied ill hnso true since microsoft want...,0.8689,0.7906,positive,positive
1,aliswe,23989758,Yes that being said the demo is an old version.,1596046007,comment,yes said demo old version,0.4019,0.4019,positive,positive
2,DeonPenny,23989759,I&#x27;d do 27 months for millions of dollars....,1596046009,comment,months millions dollars might even thousands d...,0.0000,0.0000,positive,positive
4,msla,23989761,I just described how Minitel was completely di...,1596046021,comment,described minitel completely different interne...,0.0000,0.0000,positive,positive
7,pc86,23989764,"&gt; See the problem now?<p>No, because I am a...",1596046032,comment,see problem free speech advocate thing free sp...,0.7602,0.9040,positive,positive
8,dheera,23989765,"Honest question here. Why are we, the taxpayer...",1596046041,comment,honest question taxpayers paying months free h...,0.8951,0.6808,positive,positive
9,UncleMeat,23989766,&gt; You have to bake the security into your w...,1596046041,comment,bake security workflows processes code want ma...,0.4019,0.4215,positive,positive
10,gmfawcett,23989767,Sorry to hear about the flashbacks. Having wri...,1596046044,comment,sorry hear flashbacks written couple dreadful ...,0.8244,0.2263,positive,positive
11,beervirus,23989768,Strange. Works beautifully on iOS.,1596046049,comment,strange works beautifully ios,0.4404,0.4404,positive,positive
12,awillen,23989769,You&#x27;re obviously coming in with an anti-P...,1596046054,comment,obviously coming anti bent interested pull sta...,0.7003,0.5423,positive,positive


In [ ]:
df_comments.to_csv('hn_sentiments.csv')

In [ ]:
import pandas as pd
import json
import urllib
import requests

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
def get_compund_score(text):
        score = analyzer.polarity_scores(text)
        str(text)
        return score['compound']

In [10]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

In [11]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
import pandas as pd
import json
import urllib
import requests

def get_score(entries):
  data = []
  for id in entries:
    html = urllib.request.urlopen('https://hacker-news.firebaseio.com/v0/item/' + str(id) + '.json')
    data.append(json.loads(html.read()))

  print(data[0])
  data = [i for i in data if i is not None]

  df = pd.DataFrame.from_dict(data)
  df_comments = df[df['type'] == 'comment']

  df_comments['clean_text']=df_comments['text'].map(lambda s:preprocess(s)) 
  df_comments['clean_vader_score'] = df_comments['clean_text'].apply(get_compund_score)

  return df_comments['clean_vader_score'].sum() #we can use mean()

  
 

In [35]:
import pandas as pd
import json
import urllib
import requests

def get_cummulative_score(username):

  data = []
  html = urllib.request.urlopen('https://hacker-news.firebaseio.com/v0/user/' + str(username) + '.json?print=pretty')
  data.append(json.loads(html.read()))
  df2 = pd.DataFrame.from_dict(data)
  entries =  (df2['submitted'][0])

  score = get_score(entries)
  return score


    

  



In [36]:

score = get_cummulative_score('jl')
print("Cummulative score for user ", score )

{'by': 'jl', 'descendants': 536, 'id': 19464269, 'kids': [19464832, 19465697, 19465135, 19464960, 19465786, 19465019, 19464652, 19466183, 19470050, 19465299, 19465037, 19467582, 19466713, 19466536, 19467580, 19465893, 19466505, 19464708, 19465389, 19467096, 19466016, 19465003, 19465088, 19469424, 19468661, 19468811, 19475637, 19471240, 19468924, 19465592, 19469386, 19466312, 19467740, 19466241, 19466112, 19465889, 19467249, 19475393, 19465474, 19465638, 19466066, 19466993, 19465036, 19467011, 19467899, 19465798, 19466036, 19465314, 19465127, 19465038, 19467292, 19464560], 'score': 268, 'time': 1553272932, 'title': 'Women: Learn to Program This Summer', 'type': 'story', 'url': 'http://foundersatwork.posthaven.com/women-learn-to-program-this-summer'}
Cummulative score for user  39.2382


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [39]:
test_message = "This is really bad. Never going there again. Absolute worst"
print (get_compund_score(test_message))

test_message = "This is really good. Loved the place. Employees were super kind. Can't wait to go back again."
print (get_compund_score(test_message))



-0.8357
0.94
